In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import os

Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 75
data_augmentation = True
num_predictions = 10

In [3]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [8]:
from keras import models, layers
from keras.initializers import lecun_normal

def build_student():
    initializer = lecun_normal(seed=0)

    model = models.Sequential()
    model.add(layers.Flatten(input_shape=(32, 32, 3)))
    model.add(layers.Dense(64, kernel_initializer=initializer))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dense(64, kernel_initializer=initializer))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dense(10, kernel_initializer=initializer))
    model.add(layers.Activation('softmax'))
    
    return model

In [9]:
student = build_student()
# initiate optimizer
opt = 'adam'
student.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator()

In [10]:
from keras.callbacks import EarlyStopping

datagen.fit(x_train)
history = student.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                      epochs=epochs,
                      validation_data=(x_test, y_test),
                               callbacks=[
                                        EarlyStopping(monitor='val_loss', patience=5, min_delta=0.005)
                                    ])


Epoch 1/75
391/391 [==============================] - 5s 14ms/step - loss: 1.6882 - accuracy: 0.4031 - val_loss: 1.7398 - val_accuracy: 0.3922
Epoch 2/75
391/391 [==============================] - 4s 11ms/step - loss: 1.4783 - accuracy: 0.4762 - val_loss: 2.0654 - val_accuracy: 0.3245
Epoch 3/75
391/391 [==============================] - 4s 11ms/step - loss: 1.3925 - accuracy: 0.5067 - val_loss: 1.4973 - val_accuracy: 0.4668
Epoch 4/75
391/391 [==============================] - 4s 11ms/step - loss: 1.3339 - accuracy: 0.5273 - val_loss: 1.7251 - val_accuracy: 0.3998
Epoch 5/75
391/391 [==============================] - 4s 11ms/step - loss: 1.2888 - accuracy: 0.5417 - val_loss: 1.6074 - val_accuracy: 0.4332
Epoch 6/75
391/391 [==============================] - 4s 11ms/step - loss: 1.2522 - accuracy: 0.5537 - val_loss: 1.5195 - val_accuracy: 0.4629
Epoch 7/75
391/391 [==============================] - 4s 11ms/step - loss: 1.2219 - accuracy: 0.5631 - val_loss: 1.6201 - val_accuracy: 0.438

In [11]:
results = student.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 100us/step


In [12]:
student.summary()
print('Errors: ', len(x_test) - results[1] * len(x_test))
print('Accuracy: ', results[1]*100, '%')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                196672    
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256       
_________________________________________________________________
activation_2 (Activation)    (None, 64)               

In [13]:
student.save('models/cifar10_student_2.h5')